In [2]:
import pandas as pd
import numpy as np

## Uncensoring KPIs

In [ ]:
def uncensoring_KPI(dataframe, method_name):
    """
    Calculate KPIs for demand forecasting evaluation
    
    Parameters:
    - dataframe: pandas DataFrame containing 'Verkauf' (predicted) and 'Verkauf_MBR' (ground truth) columns
    - method_name: string name of the method being evaluated
    - censorship_pct: censorship percentage for display
    - reduction_pct: reduction percentage for display
    - alpha: weight parameter for Weighted MAE (default=1)
            α = 0: standard MAE (no weighting)
            α = 1: linear weighting by true demand
            α > 1: over-proportional penalization of larger errors
            α < 1: emphasis on smaller demands
    """
    df = dataframe[dataframe['Zensiert']==1]
    censored_fraction = len(df)/len(dataframe)
    
    # Extract predicted and true values
    y_pred = df[method_name + "_Demand"].values  # ŷᵢ (estimated demand)
    y_true = df['Verkauf_MBR'].values  # yᵢ (true demand)
    
    n = len(y_pred)
    if n == 0: return
    
    # 1. Bias calculation
    bias = np.sum(y_pred - y_true) / n
    
    # 2. Accuracy (exact matches)
    exact_matches = np.sum(y_pred == y_true)
    accuracy = exact_matches / n
    
    # 3. Overestimation Rate
    overestimations = np.sum(y_pred > y_true)
    overestimation_rate = overestimations / n
    
    # 4. Underestimation Rate
    underestimations = np.sum(y_pred < y_true)
    underestimation_rate = underestimations / n
    
    # 5. Weighted MAE
    alphas = [0, 0.5, 1, 1.5]
    weighted_maes = {}
    for a in alphas:
        if a == 0:
            # Standard MAE (no weighting)
            weighted_maes[a] = np.mean(np.abs(y_pred - y_true))
        else:
            # Weighted MAE with α parameter
            weights = np.power(y_true, a)
            # Handle case where y_true might be 0
            weights = np.where(y_true == 0, 0, weights)
            weighted_maes[a] = np.sum(weights * np.abs(y_pred - y_true)) / np.sum(weights) if np.sum(weights) > 0 else 0

    # 6. Gini Coefficient
    abs_errors = np.abs(y_pred - y_true)
    mean_abs_error = np.mean(abs_errors)
    n = len(abs_errors)
    sorted_errors = np.sort(abs_errors)
    
    weighted_sum = 0.0
    total_sum = 0.0
    
    for i in range(n):
        weighted_sum += (i + 1) * sorted_errors[i]
        total_sum += sorted_errors[i]
    
    gini_coefficient = (2 * weighted_sum) / (n * total_sum) - (n + 1) / n
    
    # 7. Overstock, out of curiosity
    overstock = np.sum(np.maximum(0, y_pred - y_true))

    for i in range(n):
        weighted_sum += (i + 1) * sorted_errors[i]
        total_sum += sorted_errors[i]

    gini_coefficient = (2 * weighted_sum) / (n * total_sum) - (n + 1) / n

    # Determine bias direction
    bias_direction = "overestimation" if bias > 0 else "underestimation" if bias < 0 else "neutral"
    
    return {
        method_name+'_bias': bias,
        method_name+'_weighted_mae_0': weighted_maes[0],
        method_name+'_weighted_mae_0.5': weighted_maes[0.5],
        method_name+'_weighted_mae_1': weighted_maes[1],
        method_name+'_weighted_mae_1.5': weighted_maes[1.5],
        method_name+'_censored_fraction': censored_fraction,
        method_name+'_accuracy': accuracy,
        method_name+'_overestimation_rate': overestimation_rate,
        method_name+'_underestimation_rate': underestimation_rate,
        method_name+'_gini_coefficient': gini_coefficient,
        method_name+'_overstock': overstock
        }

def uncensoring_KPI_POS(dataframe, methods, magazines):
    '''
    dataframe: df containing all uncensored results and original data columns
    Calculate uncensoring KPIs grouped by POS.
    Returns dataframe with KPI of each method per magazine and POS
    '''
    df = dataframe.copy()
    df=df.dropna(subset=['Verkauf', 'Bezug'])
    
    df_KPIs = pd.DataFrame() #initialize final output KPI dataframe
    for letter in magazines:
        print('-'*10+ f'Magazine {letter} KPIs' + '-'*10)
        df_magazine = df[df['Magazine']==letter]
        df_magazine_KPIs = None #initialize KPI dataframe for this magazine

        for method in methods:
            method_KPIs = [] #initialize KPIs for this magazine and method
            
            for pos, group in df_magazine.groupby('EHASTRA_EH_NUMMER'):
                #do not calculate KPI for POS with less than 3 uncensored points
                if len(group[group['Zensiert']==1])<3: continue
                pos_result = uncensoring_KPI(group, method)

                #catch None output
                if pos_result is not None:
                    pos_result['EHASTRA_EH_NUMMER'] = pos
                    method_KPIs.append(pos_result)
            df_method_KPIs = pd.DataFrame(method_KPIs)
            if len(df_method_KPIs)==0: continue
            df_method_KPIs['Magazine']=letter

            print('-'*10)
            kpi_cols = [c for c in df_method_KPIs.columns if c not in ['Magazine', 'EHASTRA_EH_NUMMER']]
            for col in kpi_cols:
                print(f'{col}: {df_method_KPIs[col].mean():.3f}')

            if df_magazine_KPIs is None: df_magazine_KPIs = df_method_KPIs
            else:
                new_cols = [c for c in df_method_KPIs if c not in df_magazine_KPIs or c in ['EHASTRA_EH_NUMMER']]
                df_magazine_KPIs = pd.merge(df_magazine_KPIs, df_method_KPIs[new_cols], how = 'outer', on = ['EHASTRA_EH_NUMMER'])
        df_KPIs = pd.concat([df_KPIs, df_magazine_KPIs])

    #move magazine and pos columns to the front
    df_KPIs.set_index(['Magazine', 'EHASTRA_EH_NUMMER'], inplace=True)
    df_KPIs.reset_index(inplace=True)
    return df_KPIs

In [25]:
df = pd.read_csv('Uncensoring_results.csv')
uncensoring_methods = ['N1', 'N2', 'N3', 'EM', 'PD', 'Nahmias', 'Conrad', 'Baseline', 'Bayesian']
magazine_letters = [letter for letter in 'ABCDEFGHI']
df_KPIs = uncensoring_KPI_POS(df, uncensoring_methods, magazine_letters)
df_KPIs.to_csv('Uncensoring_KPIs.csv', index=False) #adjust accordingly

----------Magazine A KPIs----------
----------
N1_bias: -0.577
N1_weighted_mae_0: 0.859
N1_weighted_mae_0.5: 0.898
N1_weighted_mae_1: 0.955
N1_weighted_mae_1.5: 1.029
N1_censored_fraction: 0.298
N1_accuracy: 0.320
N1_overestimation_rate: 0.091
N1_underestimation_rate: 0.589
N1_gini_coefficient: 0.406
N1_overstock: 5.833
----------
N2_bias: -0.577
N2_weighted_mae_0: 1.042
N2_weighted_mae_0.5: 1.059
N2_weighted_mae_1: 1.096
N2_weighted_mae_1.5: 1.153
N2_censored_fraction: 0.298
N2_accuracy: 0.195
N2_overestimation_rate: 0.141
N2_underestimation_rate: 0.664
N2_gini_coefficient: 0.318
N2_overstock: 9.500
----------
N3_bias: -0.218
N3_weighted_mae_0: 0.683
N3_weighted_mae_0.5: 0.659
N3_weighted_mae_1: 0.645
N3_weighted_mae_1.5: 0.640
N3_censored_fraction: 0.298
N3_accuracy: 0.425
N3_overestimation_rate: 0.141
N3_underestimation_rate: 0.434
N3_gini_coefficient: 0.471
N3_overstock: 9.500
----------
EM_bias: -0.195
EM_weighted_mae_0: 0.646
EM_weighted_mae_0.5: 0.664
EM_weighted_mae_1: 0.680
EM